# 如何解析 JSON 输出

:::info 先决条件

本指南假设您熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [输出解析器](/docs/concepts/output_parsers)
- [提示模板](/docs/concepts/prompt_templates)
- [结构化输出](/docs/how_to/structured_output)
- [链接运行器](/docs/how_to/sequence/)

:::

尽管某些模型提供商支持[返回结构化输出的内置方法](/docs/how_to/structured_output)，但并非所有提供商都支持。我们可以使用输出解析器来帮助用户通过提示指定任意 JSON schema，查询符合该 schema 的模型输出，并最终将该 schema 解析为 JSON。

:::note
请记住，大型语言模型是容易出错的抽象！您必须使用容量足够大的 LLM 来生成格式正确的 JSON。
:::

`JsonOutputParser` 是一个内置选项，用于提示并解析 JSON 输出。虽然它的功能与 `PydanticOutputParser` 类似，但它还支持流式返回部分 JSON 对象。

以下是一个将它与 [Pydantic](https://docs.pydantic.dev/) 一起使用的示例，可以方便地声明预期的 schema：

In [ ]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two tired!'}

请注意，我们正在将 `format_instructions` 从解析器直接传递到提示中。您可以并且应该尝试在提示的其他部分添加您自己的格式化提示，以增强或替换默认说明：

In [3]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'

## 流式输出

如上所述，`JsonOutputParser` 和 `PydanticOutputParser` 的一个关键区别是 `JsonOutputParser` 输出解析器支持流式输出的部分片段。具体如下：

In [4]:
for s in chain.stream({"query": joke_query}):
    print(s)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why couldn'}
{'setup': "Why couldn't"}
{'setup': "Why couldn't the"}
{'setup': "Why couldn't the bicycle"}
{'setup': "Why couldn't the bicycle stand"}
{'setup': "Why couldn't the bicycle stand up"}
{'setup': "Why couldn't the bicycle stand up by"}
{'setup': "Why couldn't the bicycle stand up by itself"}
{'setup': "Why couldn't the bicycle stand up by itself?"}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': ''}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was two'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was two tired'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline'

## 无 Pydantic

您也可以在不使用 Pydantic 的情况下使用 `JsonOutputParser`。这将提示模型返回 JSON，但不会提供关于 schema 具体应该是什么样的信息。

In [5]:
joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'response': "Sure! Here's a joke for you: Why couldn't the bicycle stand up by itself? Because it was two tired!"}

## 后续步骤

你现在已经学会了一种提示模型返回结构化 JSON 的方法。接下来，请查阅 [获取结构化输出的更广泛指南](/docs/how_to/structured_output) 来了解其他技术。